# 06wk-2: 다항분류

최규빈  
2025-01-01

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/05wk-2
    .ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [6]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-yx6DQsLACqw8pWm0udv8Jm&si=in1eMD0-wU49y7mS >}}

# 2. Imports

In [20]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [21]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. 다항분류

## A. 이항분류와 `BCEWithLogitsLoss`

`-` 데이터

In [49]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
# test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True)
to_tensor = torchvision.transforms.ToTensor()
X0_train = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==0])
X1_train = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==1])
# X0_test = torch.stack([to_tensor(Xi) for Xi, yi in test_dataset if yi==0])
# X1_test = torch.stack([to_tensor(Xi) for Xi, yi in test_dataset if yi==1])
X = torch.concat([X0_train,X1_train],axis=0).reshape(-1,784)
y = torch.tensor([0.0]*len(X0_train) + [1.0]*len(X1_train)).reshape(-1,1)
# XX = torch.concat([X0_test,X1_test],axis=0).reshape(-1,784)
# yy = torch.tensor([0.0]*len(X0_test) + [1.0]*len(X1_test)).reshape(-1,1)

-   굳이 테스트는 필요없어서..

`-` 예전코드는 아래와 같음 (sig는 수동처리함)

In [55]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1)
)
loss_fn = torch.nn.BCELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,11):
    # step1 
    netout = net(X) # netout = logits 
    yhat = torch.exp(netout) / (1 + torch.exp(netout)) # yhat = prob 
    # step2
    loss = loss_fn(yhat,y) 
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()
    #---에폭끝나고 확인할 것들---#
    acc = ((net(X).data > 0)  == y).float().mean()
    print(f"epoch = {epoc}\t acc = {acc:.4f}")

epoch = 1    acc = 0.4677
epoch = 2    acc = 0.4677
epoch = 3    acc = 0.4745
epoch = 4    acc = 0.5572
epoch = 5    acc = 0.6995
epoch = 6    acc = 0.8175
epoch = 7    acc = 0.8904
epoch = 8    acc = 0.9255
epoch = 9    acc = 0.9444
epoch = 10   acc = 0.9577

`#` netout(= logits) 의 특징

-   $netout > 0 \Leftrightarrow sig(netout) >0.5$
-   $netout < 0 \Leftrightarrow sig(netout) <0.5$

`-` 그런데 위의 코드는 아래의 코드와 같음

In [56]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1)
)
loss_fn = torch.nn.BCEWithLogitsLoss() # <--- 여기를 바꾸고 
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,11):
    # step1 
    netout = net(X) # netout = logits 
    # yhat = torch.exp(netout) / (1 + torch.exp(netout))  # yhat = prob 
    # step2
    loss = loss_fn(netout,y) 
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()
    #---에폭끝나고 확인할 것들---#
    acc = ((net(X).data > 0)  == y).float().mean()
    print(f"epoch = {epoc}\t acc = {acc:.4f}")

epoch = 1    acc = 0.4677
epoch = 2    acc = 0.4677
epoch = 3    acc = 0.4745
epoch = 4    acc = 0.5572
epoch = 5    acc = 0.6995
epoch = 6    acc = 0.8175
epoch = 7    acc = 0.8904
epoch = 8    acc = 0.9255
epoch = 9    acc = 0.9444
epoch = 10   acc = 0.9577

## B. 범주형자료의 변환

`-` 범주형자료를 숫자로 어떻게 바꿀까?

-   실패 / 성공 $\to$ 0 / 1
-   숫자0그림 / 숫자1그림 $\to$ 0 / 1
-   강아지그림 / 고양이그림 $\to$ 0 / 1
-   강아지그림 / 고양이그림 / 토끼그림 $\to$ 0 / 1 / 2 ?????

`-` 주입식교육: 강아지그림/고양이그림/토끼그림일 경우 숫자화시키는 방법

-   잘못된방식: 강아지그림 = 0, 고양이그림 = 1, 토끼그림 = 2
-   올바른방식: 강아지그림 = \[1,0,0\], 고양이그림 = \[0,1,0\], 토끼그림
    = \[0,0,1\] \### \<– 이런방식을 원핫인코딩이라함

`-` 왜?

-   설명1: 강아지그림, 고양이그림, 토끼그림은 서열측도가 아니라
    명목척도임. 그래서 범주를 0,1,2 로 숫자화하면 평균등의 의미가 없음
    (사회조사분석사 2급 스타일)
-   설명2: 범주형은 원핫인코딩으로 해야함 (“30일만에 끝내는
    실전머신러닝” 이런 책에 나오는 스타일)
-   설명3: 동전을 한번 던져서 나오는 결과는 $n=1$인 이항분포를 따름.
    주사위 한번 던져서 나오는 눈금의 숫자는 $n=1$인 다항분포를 따름.
    $n=1$인 이항분포의 실현값은 0,1 이고, $n=1$인 다항분포의 실현값은
    \[1,0,0\], \[0,1,0\], \[0,0,1\] 이므로 당연히 $y_i$ 는 \[1,0,0\],
    \[0,1,0\], \[0,0,1\] 중 하나의 형태를 가진다고 가정하는게 바람직함
    (이 설명이 이 중에서 가장 정확한 설명임)

## C. 실습: 3개의 클래스를 구분¶

`-` 데이터준비

In [125]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
to_tensor = torchvision.transforms.ToTensor()
X0 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==0])
X1 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==1])
X2 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==2])
X = torch.concat([X0,X1,X2]).reshape(-1,1*28*28)
y = torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2)).reshape(-1,1).float()

In [126]:
y = torch.nn.functional.one_hot(y.flatten().long()).float()

`-` 적합

In [127]:
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,3),
)
loss_fn = torch.nn.CrossEntropyLoss() # 이름이 좀 그래.. 나같으면 CEWithLogitsLoss 라고 했을듯 
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(100):
    ## step1 
    netout = net(X)
    ## step2 
    loss = loss_fn(netout,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

In [128]:
(net(X).argmax(axis=1)  == y.argmax(axis=1)).float().mean()

## D. 결론 – 외우세여

`-` 파이토치버전 // 코딩용

|   분류   | netout의 의미 |      손실함수       |
|:--------:|:-------------:|:-------------------:|
| 이항분류 |     prob      |      `BCELoss`      |
| 이항분류 |     logit     | `BCEWithLogitsLoss` |
| 다항분류 |     probs     |         NA          |
| 다항분류 |    logits     | `CrossEntropyLoss`  |

> `CrossEntropyLoss` 이거 이름이 완전 마음에 안들어요..
> `CEWithLogitsLoss` 라고 하는게 더 좋을 것 같습니다.

`-` 일반적개념 // 이론용

|   분류   | 오차항의가정 | 마지막활성화함수 |       손실함수       |
|:--------:|:------------:|:----------------:|:--------------------:|
| 이항분류 |   이항분포   |    sigmoid[1]    | Binary Cross Entropy |
| 다항분류 |   다항분포   |    softmax[2]    |    Cross Entropy     |

[1] prob=sig(logit)

[2] probs=soft(logits)